## WOMARS regionalization

**WOMARS** stands for **W**orldwide **M**arine **R**adioactivity **S**tudies.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Polygon, Point
from pysal.lib import weights
from libpysal.weights import w_subset
import h3

from palettable.tableau import Tableau_10
import folium
import seaborn as sns
import contextily

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import calinski_harabasz_score

from functools import partial
from pathlib import Path

# from matplotlib import colors
import matplotlib.pyplot as plt


## Data loading

In [ ]:
fname = Path.home() / 'pro/data/maris/190130 Seawater10m_Sediment_1991-2000.xlsx'
df = pd.read_excel(fname, sheet_name='Seawater_137Cs'); df.shape

In [ ]:
df.head()

### Data selection & preprocessing

In [ ]:
df['begperiod'] = pd.to_datetime(df['begperiod'])
df = df[['longitude', 'latitude', 'begperiod', 'activity,MDA/2']]
df.columns = ['lon', 'lat', 'time', 'activity']

In [ ]:
df.head()

In [ ]:
df['activity'].plot(kind='hist', logy=True,bins=20);  

In [ ]:
geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]

db = (
    (gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326"))
    .to_crs(epsg=3857)[['time', 'activity', 'geometry']]
    .dropna()
    )
db.info()

In [ ]:
# Plotting activity
f, ax = plt.subplots(1, figsize=(9, 9))
db.plot(
    column="activity",
    cmap="viridis",
    scheme="quantiles",
    k=5,
    edgecolor="white",
    linewidth=0.0,
    alpha=0.5,
    legend=True,
    legend_kwds={"loc": 2},
    ax=ax,
    markersize=3
)
contextily.add_basemap(
    ax,
    crs=db.crs,
    source=contextily.providers.CartoDB.Positron
    
)
ax.set_axis_off()

## H3 encoding

In [ ]:
# https://uber.github.io/h3-py/api_reference
# https://h3geo.org/docs/quickstart
def to_h3(row, resolution=1, lat_col='lat', lon_col='lon'): 
    return h3.latlng_to_cell(row[lat_col], row[lon_col], resolution)

In [ ]:
resolution = 3
df['h3_idx'] = df.apply(partial(to_h3, resolution=resolution), axis=1)

In [ ]:
df.head()

In [ ]:
df['h3_idx'].value_counts().plot(kind='hist', logy=True,bins=20)

In [ ]:
df['activity_log'] = np.log10(df.activity)

In [ ]:
aggregations = {
    'activity_log': [
        ('q1', lambda x: np.percentile(x, 1)),
        ('q2', lambda x: np.percentile(x, 2)),
        ('q3', lambda x: np.percentile(x, 3)),
        ('mean', 'mean'),
        ('min', 'min'),
        ('max', 'max'),
        ('std', lambda x: np.nanstd(x)),
        ('n', 'count')
    ]
}

df_hex = df.groupby('h3_idx').agg(aggregations); df_hex


In [ ]:
# Reset the column names for the result DataFrame if needed
# result.columns = [f'{col[0]} ({col[1]})' for col in result.columns]
df_hex.columns = [f'{col[1]}' for col in df_hex.columns]
df_hex.reset_index(inplace=True)

In [ ]:
df_hex.head()

In [ ]:
df_hex.n.plot(kind='hist', logy=True,bins=20);

In [ ]:
def h3_to_polygon(h3_idx):
    h3_idx_geojson = h3.cell_to_boundary(h3_idx, geo_json=True)
    polygon = Polygon(h3_idx_geojson)
    return polygon

In [ ]:
df_hex['geometry'] = df_hex['h3_idx'].apply(h3_to_polygon)

In [ ]:
df_hex.head()

## To Geopandas

In [ ]:
gdf = gpd.GeoDataFrame(df_hex, geometry='geometry', crs="EPSG:4326")

In [ ]:
gdf.head()

In [ ]:
# gdf = gdf.to_crs(3857)

In [ ]:
gdf.columns

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot(ax=ax, column='q2', cmap='OrRd', scheme='quantiles', 
         legend=True, legend_kwds={"loc": 2})

ax.set_title("Mean Activity (Log10)")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude");

## Regionalization

### Spatial weights

#### Contiguity

In [ ]:
wq = weights.contiguity.Queen.from_dataframe(gdf)

In [ ]:
#wq.islands

#### K-nn

In [ ]:
wknn = weights.distance.KNN.from_dataframe(gdf, k=6)

#### Merge (where islands)

In [ ]:
def merge(wq, wknn):
    neighbors = wq.neighbors.copy()
    for i in wq.islands:
        neighbors[i] = list(wknn[i].keys())
    return weights.W(neighbors)

w = merge(wq, wknn)

#### Get disconnected components

In [ ]:
idx_cpt = lambda i: np.where(w.component_labels == i)[0]
w_cpt = lambda i: w_subset(w, idx_cpt(i))

w_cpt(1).neighbors

In [ ]:
w_cpt(4).neighbors

## Clustering

In [ ]:
cpt = 4
max_clusters = 10
cols = ['q1', 'q2', 'q3', 'mean', 'min', 'max', 'std']


In [ ]:
X = gdf.iloc[idx_cpt(cpt)][cols].values; X.shape

In [ ]:
X = gdf.iloc[idx_cpt(cpt)][cols].values
X_t = StandardScaler().fit_transform(X)

for n in range(2, max_clusters+1):
    model = AgglomerativeClustering(
        linkage='ward', 
        connectivity=w_cpt(cpt).sparse,
        n_clusters=n
    )
    model.fit(X_t)
    
    score = calinski_harabasz_score(X_t, model.labels_)
    print(f'# clusters: {n} | score: {score}')

In [ ]:
n_clusters = 3
X = gdf.iloc[idx_cpt(cpt)][cols].values
X_t = StandardScaler().fit_transform(X)

model = AgglomerativeClustering(
    linkage='ward', 
    connectivity=w_cpt(cpt).sparse,
    n_clusters=n_clusters
)

model.fit(X_t)

### Visualize

In [ ]:
gdf_sub = gdf.iloc[idx_cpt(cpt),:].copy()
gdf_sub['cluster'] = model.labels_

In [ ]:
fig, ax = plt.subplots(1, figsize=(9, 9))
gdf_sub.plot(
    column="cluster",
    categorical=True,
    legend=True,
    linewidth=0,
    ax=ax,
    legend_kwds={'loc': 'lower right'}
)

ax.set_axis_off()

In [ ]:
def style_function(feature, n_clusters, prop_name='cluster', palette=Tableau_10):
    attribute_value = feature['properties'][prop_name]
    return {'fillColor': palette.hex_colors[attribute_value], 
            'color': 'none', 
            'fillOpacity': 0.5}

In [ ]:
gdf = gdf_sub.to_crs(epsg=4326)

m = folium.Map(location=[gdf_sub.centroid.y.mean(), 
                         gdf_sub.centroid.x.mean()], zoom_start=3)

geojson_data = gdf_sub.to_json()

folium.GeoJson(geojson_data, style_function=partial(style_function, n_clusters=n_clusters)).add_to(m)

m

In [ ]:
sns.histplot(data=gdf_sub, x='mean', hue='cluster', 
             fill=False,
             multiple='layer', 
             element='step',
             palette=Tableau_10.mpl_colors,
             kde=True, 
            #  alpha=0.6,
             line_kws={'linestyle': 'dashed', 'alpha': 1}
             )

plt.xlabel('mean')
plt.ylabel('Frequency');

In [ ]:
sns.set(style="whitegrid")
g = sns.FacetGrid(gdf_sub, row='cluster', sharex=True, 
                  aspect=4, height=1.5,
                  palette=Tableau_10.mpl_colors, hue='cluster')

# g.map(sns.kdeplot, 'mean', color='blue', alpha=0.5, fill=True)
g.map(sns.kdeplot, 'mean', alpha=0.5, fill=True)

# Set plot labels and title
g.set_axis_labels('mean', 'Density')
g.fig.subplots_adjust(top=0.8)
# g.fig.suptitle('Faceted KDE Plot with Adjusted Alpha');

### Mergin hexs within same cluster

In [ ]:
gdf_sub.head()

In [ ]:
gdf_sub[gdf_sub.cluster == 0].plot()

In [ ]:
cluster = 0
geom = gdf_sub[gdf_sub.cluster == cluster].buffer(0.001).unary_union
gdf_cluster_union = gpd.GeoDataFrame({'geometry': [geom]})
gdf_cluster_union.plot();

### Clustering on coordinates

In [ ]:
df.head()

In [ ]:
from sklearn.cluster import DBSCAN, HDBSCAN
import numpy as np
from haversine import haversine, Unit

# Example coordinates: (latitude, longitude)
# coordinates = np.array([
#     [40.748817, -73.985428],  # Empire State Building
#     [40.689247, -74.044502],  # Statue of Liberty
#     [37.4189, -122.0819],     # Googleplex
#     [37.4275, -122.1697]      # Stanford University
# ])

coordinates = df[['lat', 'lon']].to_numpy()

# Convert latitude and longitude degrees to radians
coordinates_rad = np.radians(coordinates)

In [ ]:
# Define the epsilon distance (in kilometers)
epsilon = 100 / 6371.0088  # 100 km radius, divided by the Earth's radius in km to convert to radians

# Create a DBSCAN clusterer. Note that haversine distance requires data in the form of [lat, lon] and both in radians
# clusterer = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine')

clusterer = HDBSCAN(min_cluster_size=100, metric='haversine')

# Fit the clusterer to the data
cluster_labels = clusterer.fit_predict(coordinates_rad)

# Count the number of points per cluster
unique, counts = np.unique(cluster_labels, return_counts=True)

points_per_cluster = dict(zip(unique, counts))

# Create the bar plot
plt.bar(points_per_cluster.keys(), points_per_cluster.values())

plt.xlabel('Cluster Label')
plt.ylabel('Number of Points')
plt.title('Number of Points per Cluster Label')
plt.xticks(ticks=list(points_per_cluster.keys())); 

In [ ]:
data = {
    'Coordinates': [Point(lon, lat) for lat, lon in coordinates],
    'Cluster': cluster_labels
}

gdf = gpd.GeoDataFrame(data, geometry='Coordinates')
fig, ax = plt.subplots(figsize=(10, 6))
gdf.plot(ax=ax, column='Cluster', categorical=True, legend=True, 
         legend_kwds={'bbox_to_anchor': (1, 1)}, marker='o', markersize=10)

# Adjust the plot
ax.set_title('Clustered Points');

## Classical thematic mapping